This notebook was used to clean corrupted images from the dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../code/')

In [3]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from data_loading import ImageSequence
from utils import square_pad_image

In [4]:
dataset_path = '/data/datasets/imagenet2012/'

In [5]:
train_generator = ImageSequence(dataset_path, 20, target_split='train', square_ims='center_pad')

  1%|          | 7/1001 [00:00<00:15, 63.92it/s]

generating splits


100%|██████████| 1001/1001 [00:15<00:00, 64.93it/s]


In [6]:
folders = train_generator.list_valid_folders()

In [7]:
from PIL import UnidentifiedImageError
from PIL import Image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
# import warnings
# warnings.filterwarnings('error')

In [82]:
def correct_sample(sample):
    sample['status_code'] = 'er'
    sample['fname'] = None
    return sample

def curate(folder_name, in_data='/curated_results.json'):
    if isinstance(in_data, str):
        ims_df = pd.read_json(folder_name + in_data)
    elif isinstance(in_data, pd.core.frame.DataFrame):
        ims_df = in_data
    invalids = []
    ims = []
    for i, sample in ims_df[ims_df['status_code']==200].iterrows():
        print(sample['fname'])
        try:
            if sample['fname'] is None:
                sample = correct_sample(sample)
                ims_df.iloc[i] = sample
            else:
                im = square_pad_image(sample['fname'])
                ims.append(np.asarray(im))
        except UnidentifiedImageError:
            sample = correct_sample(sample)
            ims_df.iloc[i] = sample
        except OSError:
            sample = correct_sample(sample)
            ims_df.iloc[i] = sample
        except Exception as e:
            print(e)
            raise Exception('BREAK other error')
    print(folder_name, len(ims))
    preprocess_input(np.array(ims))
    new_json_path = folder_name + '/curated_results.json'
    ims_df.to_json(new_json_path)

In [97]:
d = pd.read_json('/data/datasets/imagenet2012/brad/curated_results.json')

In [92]:
already = ['{}'.format(x) for x in range(1200)]
main_path = '/data/datasets/imagenet2012/brad/'
all_f = sorted(os.listdir(main_path))
news = [x for x in all_f if (x.split('.')[0] not in already)]
news_full = ['{}/{}'.format(main_path, x) for x in news if x.split('.')[-1] != 'json']


In [94]:
new_df = pd.DataFrame({'fname': news_full})
new_df['status_code'] = 200

In [95]:
all_df = pd.concat([d, new_df]).reset_index(drop=True)

,ind,url,fname,status_code
0,0.0,http://farm1.static.flickr.com/43/100299048_85...,/data/datasets/imagenet2012//brad/0.jpeg,200
1,1.0,http://farm1.static.flickr.com/39/100299580_df...,/data/datasets/imagenet2012//brad/1.jpeg,200
2,2.0,http://farm1.static.flickr.com/8/10090770_9569...,/data/datasets/imagenet2012//brad/2.jpeg,200
3,3.0,http://farm1.static.flickr.com/19/106208948_14...,/data/datasets/imagenet2012//brad/3.jpeg,200
4,4.0,http://farm1.static.flickr.com/40/106209220_f5...,/data/datasets/imagenet2012//brad/4.jpeg,200
...,...,...,...,...
1082,NaN,NaN,/data/datasets/imagenet2012/brad//images (5).jpeg,200
1083,NaN,NaN,/data/datasets/imagenet2012/brad//images (6).jpeg,200
1084,NaN,NaN,/data/datasets/imagenet2012/brad//images (7).jpeg,200
1085,NaN,NaN,/data/datasets/imagenet2012/brad//images (8).jpeg,200


In [26]:
# from multiprocessing import Pool

# # parallel
# pool = Pool(10)
# list(tqdm(pool.imap(curate, folders), total=len(folders)))
# pool.close()

# # single thread
# # for folder_name in tqdm(folders):
# #     curate(folder_name)

In [33]:
# extra curate
# curate('/data/datasets/imagenet2012/n04399382', '/results.json')